In [10]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 


def read_automode_24hrs_before(relayNumber):
    conn = sqlite3.connect('../database/pythonsqlite.db')
    cursor = conn.cursor()
    
    current_datetime = datetime.now() - timedelta(hours=24)
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
    cursor.execute(query, (relayNumber,))
    
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)

    if int(result_list[0][0]) == 0: # if no results then check 48 hrs before data

        current_datetime = datetime.now() - timedelta(hours=48)
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

        query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
        cursor.execute(query, (relayNumber,))
        
        rows = cursor.fetchall()
        result_list = []
        for row in rows:
            result_list.append(row)


        cursor.close()

        return result_list
    else:
        cursor.close()
        return result_list


def getAutoModeStatus():
    try:
        current_datetime = datetime.now()
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)
        # Format datetime with 'T'
        start_of_day_str = start_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
        end_of_day_str = end_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]

        # Connect to the SQLite database
        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")

        df = pd.read_sql_query(f"SELECT start_timestamp, end_timestamp, marketprice, unit, relaynumber FROM automaterelay WHERE start_timestamp BETWEEN '{start_of_day_str}' AND '{end_of_day_str}'", conn)

        # Convert DataFrame to JSON
        json_data = df.to_json(orient='records')

        print(df)

        # Return JSON response
        return json_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"status": "Error"}), 500

    finally:
          if 'conn' in locals() and conn:
            conn.close()

read_automode_24hrs_before(2)


Current working directory: /home/pi/smart_plug/python_files


[(11,)]